In [1]:
#Step1: "pip install numpy pegasus-wms.api"


In [2]:
import numpy as np
from Pegasus.api import *
import time

In [3]:
%%writefile merge_sort_script.py
import sys
import numpy as np
import time

def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        L = arr[:mid]
        R = arr[mid:]

        merge_sort(L)
        merge_sort(R)

        i = j = k = 0
        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1

if __name__ == "__main__":
    arr = np.fromstring(sys.argv[1], sep=',')
    start_time = time.time()
    merge_sort(arr)
    end_time = time.time()
    print(f"Merge sort time: {end_time - start_time} seconds")


Overwriting merge_sort_script.py


In [4]:
%%writefile bubble_sort_script.py
import sys
import numpy as np
import time

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]

if __name__ == "__main__":
    n_iteration = int(sys.argv[2])
    results = []
    for i in range(n_iteration):
        arr = np.random.randint(1, 10000, 10000)
        start_time = time.time()
        bubble_sort(arr)
        end_time = time.time()
        results.append(end_time - start_time)
    
    avg_time = sum(results) / n_iteration
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    output = f"Function run time: {avg_time} seconds\nCurrent time: {current_time}"
    print(output)


Overwriting bubble_sort_script.py


In [ ]:
from Pegasus.api import *

props = Properties()
props["pegasus.monitord.encoding"] = "json"
props.write()

tc = TransformationCatalog()
merge_sort_exe = Transformation(
    "merge_sort",
    site="local",
    pfn="python C:/Users/frank/Documents/GitHub/Jup2Kub/example/pegasus/merge_sort_script.py",
    is_stageable=True
)
bubble_sort_exe = Transformation(
    "bubble_sort",
    site="local",
    pfn="python C:/Users/frank/Documents/GitHub/Jup2Kub/example/pegasus/bubble_sort_script.py",
    is_stageable=True
)
tc.add_transformations(merge_sort_exe, bubble_sort_exe)

wf = Workflow("sorting_workflow")
wf.add_transformation_catalog(tc)


random_numbers = File("random_numbers.txt")
merge_job = Job(merge_sort_exe)\
    .add_args(random_numbers, 3)\
    .add_inputs(random_numbers)

bubble_job = Job(bubble_sort_exe)\
    .add_args(random_numbers, 3)\
    .add_inputs(random_numbers)

wf.add_jobs(merge_job, bubble_job)

wf.plan(submit=True)
